# Atención primaria España

In [10]:
import pandas as pd
import os
import json

datos [porta estadístico](https://pestadistico.inteligenciadegestion.mscbs.es/publicoSNS/S/base-de-datos-de-clinicos-de-atencion-primaria-bdcap)

# consulta de datasets

In [2]:
psico_prim = pd.read_excel('../data/at_primaria/detalles_psico.xlsx')

In [3]:
psico_prim.columns = psico_prim.columns.str.replace(" ",'_')

In [4]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Grandes_Grupos,Problema_de_Salud,Casos
160047,2018,03-Asturias,Mujer,<18.000 ? /año,15-34 años,P20 - Trastornos de la memoria,429.0
188543,2019,03-Asturias,Hombre,<18.000 ? /año,65 y más años,P04 - Sensación/sentimientos de irritabilidad/...,506.0
15829,2012,12-Galicia,Mujer,<18.000 ? /año,15-34 años,P77 - Suicidio/intento de suicidio,NaN
170213,2018,09-Cataluña,Mujer,18.000-99.999 ? /año,15-34 años,P24 - Problemas específicos del aprendizaje,NaN
22534,2013,03-Asturias,Mujer,?100.000 ? /año,65 y más años,P07 - Disminución del deseo sexual,NaN


In [5]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Grandes_Grupos            0
Problema_de_Salud         0
Casos                182993
dtype: int64

In [6]:
psico_prim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213840 entries, 0 to 213839
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Año                213840 non-null  int64  
 1   CCAA               213840 non-null  object 
 2   Sexo               213840 non-null  object 
 3   Nivel_Renta        213840 non-null  object 
 4   Grandes_Grupos     213840 non-null  object 
 5   Problema_de_Salud  213840 non-null  object 
 6   Casos              30847 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 11.4+ MB


In [7]:
psico_prim.columns

Index(['Año', 'CCAA', 'Sexo', 'Nivel_Renta', 'Grandes_Grupos',
       'Problema_de_Salud', 'Casos'],
      dtype='object')

## cambiamos algunos datos categóricos

In [8]:
psico_prim[["CodCA","CCAA"]] = psico_prim.CCAA.str.split("-",expand=True)

In [ ]:
# usamos geojson de comunidades para poner el nombre a las comunidades

In [11]:
#diccionario
geo_json = r"../data/socioecon/comunidades.geojson"
with open (geo_json) as geo_file:
    comunidades = json.load(geo_file)

In [15]:
comunidades['features'][0]['properties']

{'cod_ccaa': '07',
 'noml_ccaa': 'COMUNIDAD DE CASTILLA Y LEON',
 'name': 'Castilla-Leon',
 'cartodb_id': 7,
 'created_at': '2014-09-30T00:00:00Z',
 'updated_at': '2014-12-25T02:07:41Z'}

In [22]:
cod_com = {}
for i in range(19):    
    cod_com[comunidades['features'][i]['properties']['cod_ccaa']] = comunidades['features'][i]['properties']['name']

In [25]:
psico_prim['CCAA'] = psico_prim.CodCA.map(cod_com)

In [27]:
list(psico_prim.Nivel_Renta.unique())

['Sin Clasificar',
 '?100.000 ? /año',
 '18.000-99.999 ? /año',
 '<18.000 ? /año',
 'Muy baja']

In [28]:
renta ={'Sin Clasificar':'desconocida',
 '?100.000 ? /año': '+ 100000',
 '18.000-99.999 ? /año': '18000-99999',
 '<18.000 ? /año':'- 18000',
 'Muy baja':'muy_baja'}

In [29]:
psico_prim.Nivel_Renta = psico_prim.Nivel_Renta.replace(renta)

In [30]:
psico_prim.Grandes_Grupos = psico_prim.Grandes_Grupos.str.extract(r'(\d+-*\d+)')
psico_prim.rename({"Grandes_Grupos":"Edad"},axis = 1,inplace = True)
psico_prim.Edad.replace('65','65 +', inplace = True)

In [31]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Problema_de_Salud,Casos,CodCA
114639,2016,Extremadura,Hombre,muy_baja,00-14,P20 - Trastornos de la memoria,NaN,11
91412,2015,Madrid,Mujer,+ 100000,35-64,P28 - Incapacidad/minusvalía mental,NaN,13
206044,2019,Madrid,Hombre,desconocida,65 +,P80 - Trastornos de la personalidad,NaN,13
186835,2019,Aragon,Hombre,muy_baja,00-14,P11 - Problemas de la conducta alimentaria en ...,NaN,02
160440,2018,Asturias,Mujer,desconocida,65 +,P17 - Abuso del tabaco,NaN,03


In [32]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Edad                      0
Problema_de_Salud         0
Casos                182993
CodCA                     0
dtype: int64

In [33]:
psico_prim.Casos.fillna(0.,inplace = True)

In [34]:
psico_prim[["cod_salud","problema_salud"]]= psico_prim.Problema_de_Salud.str.split(" - ",expand=True)

In [35]:
psico_prim.drop('Problema_de_Salud',axis=1,inplace = True)

## equivalencias codigos diagnósticos con CEI10

In [36]:
trans_cod = {'P01': 'F40-49','P02': 'F40-49','P03': 'F40-49','P04': 'F40-49','P05':'F00-F09','P06': 'F30-39','P07': 'F30-39',
'P08': 'F30-39','P09': 'F30-39','P09-07': 'F30-39','P10': 'F90-F98','P11': 'F90-F98','P12': 'F90-F98','P13': 'F90-F98',
'P15': 'F10-F19','P16': 'F10-F19','P17': 'F10-F19','P18': 'F10-F19','P19': 'F10-F19','P20':'F00-F09','P22+23': 'F90-F98',
'P24': 'F80-89','P25': 'F60-69','P27': 'F99','P28': 'F70-79','P29': 'F99','P70':'F00-F09','P71': 'F20-29','P72': 'F20-29',
'P73': 'F20-29','P74': 'F40-49','P75': 'F40-49','P76': 'F30-39','P77': 'F30-39','P78': 'F40-49','P79': 'F40-49','P80': 'F60-69',
'P81': 'F90-F98','P82': 'F40-49','P82/02': 'F40-49','P85': 'F70-79','P86': 'F50-59','P98': 'F20-29','P99': 'F99',
}

In [37]:
psico_prim['cei10'] = psico_prim.cod_salud.map(trans_cod)

In [38]:
psico_prim.sample()

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10
131823,2017,Baleares,Hombre,- 18000,65 +,306.0,04,P99,Otros problemas psicológicos/mentales,F99


## tipo columna casos

In [39]:
psico_prim.Casos =psico_prim.Casos.astype('int64')

In [40]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10
6935,2012,Canarias,Mujer,+ 100000,65 +,0,05,P71,Otras psicosis orgánicas,F20-29
75419,2015,Asturias,Mujer,muy_baja,65 +,0,03,P04,Sensación/sentimientos de irritabilidad/enojo,F40-49
31932,2013,Cataluña,Hombre,18000-99999,15-34,0,09,P76,Depresión/trastornos depresivos,F30-39
28940,2013,Castilla-Leon,Hombre,muy_baja,15-34,0,07,P76,Depresión/trastornos depresivos,F30-39
139839,2017,Castilla-La Mancha,Mujer,desconocida,15-34,0,08,P08,Ausencia/pérdida de la satisfación sexual,F30-39


In [41]:
psico_prim.to_csv('../data/psico_atprimaria.csv',index=False)